In [40]:
import ipywidgets as widgets


text = widgets.Text(
    value='Hello World',
    placeholder='Type something',
    description='Something really long:',
    disabled=False,
    layout=widgets.Layout(width='600px'),
    style={'description_width': 'initial'}
)
display(widgets.HBox([text], layout=widgets.Layout(border='solid')))

In [57]:
items= [widgets.Label(str(i)) for i in range(8)]
widgets.GridBox(items, layout=widgets.Layout(grid_template_columns="repeat(2, 475px)",border='solid'))

GridBox(children=(Label(value='0'), Label(value='1'), Label(value='2'), Label(value='3'), Label(value='4'), La…

In [1]:
import ipywidgets as widgets


button1 = widgets.Button(description='button1',disabled=True)
display(button1)

Button(description='button1', disabled=True, style=ButtonStyle())

In [10]:
lo = button1.layout

In [11]:
lo.visibility=False

TraitError: The 'visibility' trait of a Layout instance expected any of ['visible', 'hidden', 'inherit', 'initial', 'unset'] (case-insensitive) or None, not the bool False.

In [24]:
import ipywidgets as widgets


button2 = widgets.Button(description='button2',disabled=False)
display(button2)

Button(description='button1', style=ButtonStyle())

In [31]:
box = widgets.HBox([button1, button2],disabled=True)
box.disabled=True
display(box)

In [35]:
for elem in box.children:
    elem.disabled=False

In [5]:
from ipywidgets import FloatSlider
from IPython.display import display
confidence_slider = FloatSlider(
    value=7.5,
    min=0,
    max=1.0,
    step=0.01,
    description='Confidence:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)
display(confidence_slider)

FloatSlider(value=1.0, continuous_update=False, description='Confidence:', max=1.0, step=0.01)

In [7]:
confidence_slider.value

0.89

In [12]:
from utils.classifier import Classifier
from sklearn.neural_network import MLPClassifier
from utils.data_item import DataItem


def test_classifier():
    labeled_data=[]
    for line in open('labeled_data.csv').read().splitlines()[1:]:
        id_,label = line.split(';')
        item = DataItem(id_)
        if label=='R':
            item.set_relevant()
        else:
            item.set_irrelevant()
            assert label=='I'
        if item.has_vector():
            labeled_data.append(item)
            
    for type_ in [DataItem.TYPE_BOW, DataItem.TYPE_GLOVE300, DataItem.TYPE_GLOVE600, ]:
        print(f'Working on {type_}')
        clf = Classifier(model=MLPClassifier(),type_=type_)
        assert not clf.trained
        clf.fit(labeled_data)
        yhat = clf.predict(labeled_data)

        clf.partial_fit(labeled_data)
    print('OK!')
test_classifier()    

Working on B
Working on G3


/home/ec2-user/SageMaker/.conda/envs/imm/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Working on G6
OK!


In [7]:
yhat = clf.predict(labeled_data)

In [9]:
import numpy as np
np.sum(yhat)

620.52313

In [11]:
def test_term_highlighter():
    labeled_data=[]
    for line in open('labeled_data.csv').read().splitlines()[1:]:
        id_,label = line.split(';')
        item = DataItem(id_)
        if label=='R':
            item.set_relevant()
        else:
            item.set_irrelevant()
            assert label=='I'
        if item.has_vector():
            labeled_data.append(item)

    keep_top=20
    terms_per_article=10
    highlighter = TermHighlighter(keep_top=keep_top,terms_per_article=terms_per_article)
    assert not highlighter.trained
    assert len(highlighter.vocab)==10000
    
    highlighter.fit(labeled_data)
    
    assert len(highlighter.term2coef)==20
    assert len(highlighter.sorted_terms())==len(highlighter.vocab)
    
    assert all([elem1==elem2 for elem1, elem2 in zip(highlighter.term2coef, highlighter.sorted_terms()[:keep_top])])
    
    highlighter.highlight(' '.join(highlighter.sorted_terms()[:keep_top])) #first 'terms_per_articles' should be highlighted
    print('OK!')
    
test_term_highlighter()

OK!


In [5]:
all([elem1==elem2 for elem1,elem2 in zip(list(highlighter.term2coef), highlighter.sorted_terms())])

True

In [7]:
highlighter.highlight(' '.join(highlighter.sorted_terms()[:20]))

'<mark style="background-color:red">dp</mark> <mark style="background-color:red">title</mark> <mark style="background-color:red">person</mark> <mark style="background-color:red">canada</mark> <mark style="background-color:red">arrive</mark> <mark style="background-color:red">display</mark> <mark style="background-color:red">unch</mark> <mark style="background-color:red">low</mark> <mark style="background-color:red">oil</mark> <mark style="background-color:red">miner</mark> iro company member ltd come sale try old domestic contract'

In [4]:
len(highlighter.sorted_terms())

10000

In [8]:
str(DataItem('1287139125'))
dir(DataItem('1287139125'))


item.

<bound method DataItem.preload_vector of <utils.data_item.DataItem object at 0x7f0caf49c700>>

In [20]:
from utils.data_item import DataItem
def data_item_tester():
    item0 = DataItem('1287139125')
    assert item0.id_=='1287139125'
    assert item0.source=='GM1'
    assert item0.filename() == '/home/ec2-user/SageMaker/data/GM_all_1945_1956/1287139125.xml'
    assert item0.has_vector
    assert not item0.is_relevant()
    item0.set_irrelevant()
    assert not item0.is_relevant()
    item0.set_relevant()
    assert item0.is_relevant()    
    item0.set_irrelevant()
    assert not item0.is_relevant()
    
    vec = item0.vector(DataItem.TYPE_BOW)
    assert vec.shape==(10000,)
    vec = item0.vector(DataItem.TYPE_GLOVE300)
    assert vec.shape==(300,)
    vec = item0.vector(DataItem.TYPE_GLOVE600)
    assert vec.shape==(600,)
    print('OK!')
    
    item0.preload_vector(type_=DataItem.TYPE_BOW)
    assert len(item0._preloaded_vector)==1
    assert all(item0._preloaded_vector[DataItem.TYPE_BOW]==item0.vector(DataItem.TYPE_BOW))
    item0.preload_vector(type_=DataItem.TYPE_GLOVE300)
    assert len(item0._preloaded_vector)==2
    assert all(item0._preloaded_vector[DataItem.TYPE_GLOVE300]==item0.vector(DataItem.TYPE_GLOVE300))
    item0.preload_vector(type_=DataItem.TYPE_GLOVE600)
    assert len(item0._preloaded_vector)==3
    assert all(item0._preloaded_vector[DataItem.TYPE_GLOVE600]==item0.vector(DataItem.TYPE_GLOVE600))
    
    item0.get_htmldocview()
    
    item1 = DataItem('1269992885')
    assert item1.id_=='1269992885'
    assert item1.source=='GM2'
    
    
    item2 = DataItem('1269992885','GM2')
    
    assert item1==item2
    
    
    
    X = DataItem.get_X([item0, item1,item2],type_=DataItem.TYPE_BOW)
    assert X.shape==(3,10000)
    assert np.sum(X[0,:]-item0.vector(type_=DataItem.TYPE_BOW))<1e-5, np.sum(X[0,:]-item0.vector(type_=DataItem.TYPE_BOW))
    assert np.sum(X[1,:]-item1.vector(type_=DataItem.TYPE_BOW))<1e-5
    assert np.sum(X[2,:]-item2.vector(type_=DataItem.TYPE_BOW))<1e-5

    X = DataItem.get_X([item0, item1,item2],type_=DataItem.TYPE_GLOVE300)
    assert X.shape==(3,300)
    assert np.sum(X[0,:]-item0.vector(type_=DataItem.TYPE_GLOVE300))<1e-5
    assert np.sum(X[1,:]-item1.vector(type_=DataItem.TYPE_GLOVE300))<1e-5
    assert np.sum(X[2,:]-item2.vector(type_=DataItem.TYPE_GLOVE300))<1e-5
    
    
    X = DataItem.get_X([item0, item1,item2],type_=DataItem.TYPE_GLOVE600)
    assert X.shape==(3,600)
    assert np.sum(X[0,:]-item0.vector(type_=DataItem.TYPE_GLOVE600))<1e-5
    assert np.sum(X[1,:]-item1.vector(type_=DataItem.TYPE_GLOVE600))<1e-5
    assert np.sum(X[2,:]-item2.vector(type_=DataItem.TYPE_GLOVE600))<1e-5
    
    
    # get_y
    item0.set_irrelevant()
    item1.set_relevant()
    item2.set_irrelevant()
    
    y = DataItem.get_y([item0,item1,item2])
    assert all(y==[0, 1, 0])
    
    
    item0.set_relevant()
    item1.set_relevant()
    item2.set_irrelevant()
    y = DataItem.get_y([item0,item1,item2])
    assert all(y==[1, 1, 0])
    
    
data_item_tester()

OK!


In [6]:
import numpy as np
np.max(item1.vector(type_=DataItem.TYPE_BOW))

1.0

In [5]:
np.sum(X[1,:])

4.5

In [8]:
!ls /home/ec2-user/SageMaker/data/GM_all_1957-1967/ | head

1269992885.xml
1269992886.xml
1269992887.xml
1269992888.xml
1269992889.xml
1269992890.xml
1269992891.xml
1269993007.xml
1269993008.xml
1269993010.xml
ls: write error: Broken pipe
